# Demonstration of a Fourier series representation of a symmectric rectangular pulse train

This demo is supposed to visualize the [Fourier series](https://en.wikipedia.org/wiki/Fourier_series) representation of a periodic [rectangular pulse](https://en.wikipedia.org/wiki/Rectangular_function) train (or [square wave](https://en.wikipedia.org/wiki/Square_wave) ). 

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on signals and systems hold at the [university of applied siences, Berlin](https://www.htw-berlin.de/).

## Import libararies

In [1]:
# notebook or inline or qt
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

plt.style.use('noelle.mplstyle')

## Plot time signal and corresponding amplitudes of the Fourier coefficients

$???$

In [2]:
def fSeries(T0=1, T=2, N=5):
    """Calc Fourier series coefficients of a periodic, symmetric rectangular pulse train
    and plot them.
    
    N number of harminics to include
    T0 width of the individual rect pulses
    T period of rect pulse train
    """

    duration = 5
    dutyCycle = T0/T
    f0 = 1/T
    w0 = 2 * np.pi * f0

    sr = 500
    t = np.arange(0, duration-1/sr, 1/sr)

    rect = np.zeros_like(t)
    rect[np.mod(t, T) <= T0/2] = 1
    rect[np.mod(t, T) >= T-T0/2] = 1

    approx = 1j * np.zeros_like(rect)
    freqs = np.arange(-N + 1, N)
    X = 1j * np.zeros_like(freqs)
    
    for i, freq in enumerate(freqs):
        X[i] = dutyCycle * np.sinc(freq * dutyCycle)
        approx += X[i] * np.exp(1j * freq * w0 * t)        
    
    n_row = 1
    n_col = 2
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig, ax = plt.subplots(n_row, n_col, figsize = fig_size)
    ax[0].plot(t, rect, t, np.real(approx))
    ax[0].text(0.5, 1.2, 'duty cycle: %1.2f, N: %d' %(dutyCycle, N))
    ax[0].set(xlabel='time / s', ylim=(-0.3, 1.3), ylabel='x(t)')
    ax[0].grid(True)
    
    ax[1].stem(freqs, np.abs(X), use_line_collection=True)        
    ax[1].set(xlabel='k', ylim=(0, np.max(np.abs(X)) * 1.1), ylabel='|X[k]|')
    ax[1].grid(True)

    plt.show()
    
widgets.interact(fSeries, T0=(0.1, 2), T=(0.1, 5), N=(1, 99, 1));

interactive(children=(FloatSlider(value=1.0, description='T0', max=2.0, min=0.1), FloatSlider(value=2.0, descr…